<a href="https://colab.research.google.com/github/dnaae/minor-internship/blob/FISH-QUANT-pipeline/Spot_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install big-fish

In [ ]:
import os
import numpy as np
import tifffile as tiff
from google.colab import drive
import bigfish
import bigfish.stack as stack
import bigfish.detection as detection
import bigfish.plot as plot
from google.colab import drive
import matplotlib.pyplot as plt
import re

In [ ]:
# Function to mount Google Drive and select a folder
def choose_folder_colab():
    drive.mount('/content/drive')
    drive_folder = "/content/drive/MyDrive/"
    contents = os.listdir(drive_folder)

    print("Contents of your Google Drive:")
    for i, item in enumerate(contents):
        print(f"{i + 1}: {item}")

    while True:
        choice = input("Enter the number of the folder you want to select (q to quit): ")
        if choice.lower() == 'q':
            return None
        try:
            choice = int(choice)
            if 1 <= choice <= len(contents):
                selected_folder = os.path.join(drive_folder, contents[choice - 1])
                return selected_folder
            else:
                print("Invalid choice. Please try again or enter 'q' to quit.")
        except (ValueError, KeyboardInterrupt):
            print("Invalid input. Please try again or enter 'q' to quit.")

In [ ]:
def process_tiff_image(filename,channel,cell_name):
    # Create the full path to the current image
    path = os.path.join(path_input, filename)

    # Load the image
    rna = stack.read_image(path)
    print(f"Processing image: {filename} (Channel {channel})")
    print("smfish channel")
    print("\r shape: {0}".format(rna.shape))
    print("\r dtype: {0}".format(rna.dtype))

    rna_mip = stack.maximum_projection(rna)
    #rna_mip=stack.gaussian_filter(rna_mip, sigma=5) #apply gaussian filter
     #attempt to further denoise
    rna_log = stack.log_filter(rna_mip, sigma=3)
    #rna_background_mean = stack.remove_background_mean(rna_mip, kernel_shape="square", kernel_size=31)
    #rna_background_gaussian = stack.remove_background_gaussian(rna_mip, sigma=3)
    #images = [rna_log, rna_background_gaussian, rna_background_mean]
    print("smfish channel (2D maximum projection)")
    print("\r shape: {0}".format(rna_mip.shape))
    print("\r dtype: {0}".format(rna_mip.dtype))

    spot_detection(rna, rna_mip, channel, cell_name, path_input)

In [ ]:
#if everything fails reruns with these paraameters for FULV   log_kernel_size=(0.95,1.25, 1.25),
        #minimum_distance=(0.95, 1.25, 1.25) and image=rna,
        # spots=spots,
        # voxel_size=(300, 103, 103),
        # spot_radius=(350, 150, 150),
        # alpha=0.6, #0.6
        # beta=1, #1.3
        # gamma=5)

spots_post_clustering = None
clusters= None
def spot_detection(rna, rna_mip, channel, cell_name, path_input):
    # # Spot detection
    # spots, threshold = detection.detect_spots(
    #     images=rna,
    #     return_threshold=True,
    #     voxel_size=(300, 103, 103),
    #     spot_radius=(350, 150, 150)
    # )
    # print("detected spots")
    # print("\r shape: {0}".format(spots.shape))
    # print("\r dtype: {0}".format(spots.dtype))
    # print("\r threshold: {0}".format(threshold))
    # spot_radius_px = detection.get_object_radius_pixel(
    #   voxel_size_nm=(500, 203, 203),
    #   object_radius_nm=(450, 250, 250),
    #   ndim=3)
    # print("spot radius (z axis): {:0.3f} pixels".format(spot_radius_px[0]))
    # print("spot radius (yx plan): {:0.3f} pixels".format(spot_radius_px[-1]))
    # Kernel size and minimal distance
    # spot_radius_px = detection.get_object_radius_pixel(
    #     voxel_size_nm=(300, 103, 103),
    #     object_radius_nm=(350, 150, 150),
    #     ndim=3
    # )

    # print("spot radius (z axis): {:0.3f} pixels".format(spot_radius_px[0]))
    # print("spot radius (yx plan): {:0.3f} pixels".format(spot_radius_px[-1]))

#  filter
#     rna_log = stack.log_filter(rna, sigma=spot_radius_px)
#     #rna_log=stack.gaussian_filter(rna_mip, sigma=5)
# # local maximum detection
#     mask = detection.local_maximum_detection(rna_log, min_distance=spot_radius_px)

# # thresholding
#     threshold = detection.automated_threshold_setting(rna_log, mask)
#     spots, _ = detection.spots_thresholding(rna_log, mask, threshold)
#     print("detected spots")
#     print("\r shape: {0}".format(spots.shape))
#     print("\r dtype: {0}".format(spots.dtype))
#     print("\r threshold: {0}".format(threshold))
    spots, threshold = detection.detect_spots(
        images=rna,
        return_threshold=True,
        # log_kernel_size=(0.8, 1.15, 1.15),
        # minimum_distance=(0.8, 1.15, 1.15)
        log_kernel_size=(1.167, 1.456, 1.456),
        minimum_distance=(1.167, 1.456, 1.456)
    )

    print("detected spots")
    print("\r shape: {0}".format(spots.shape))
    print("\r dtype: {0}".format(spots.dtype))
    print("\r threshold: {0}".format(threshold))

    # Plot spot detection
    plot.plot_detection(rna_mip, spots, contrast=True)

    # Plot elbow curve
    plot.plot_elbow(
        images=rna,
        voxel_size=(300, 103, 103),
        spot_radius=(350, 150, 150))

    # Dense region decomposition
    spots_post_decomposition, dense_regions, reference_spot = detection.decompose_dense(
        image=rna,
        spots=spots,
        voxel_size=(300, 103, 103),
        spot_radius=(350, 150, 150),
        alpha=0.6, #0.6
        beta=1, #1
        gamma=5)

    print("detected spots before decomposition")
    print("\r shape: {0}".format(spots.shape))
    print("\r dtype: {0}".format(spots.dtype), "\n")
    print("detected spots after decomposition")
    print("\r shape: {0}".format(spots_post_decomposition.shape))
    print("\r dtype: {0}".format(spots_post_decomposition.dtype))

    # Plot spot detection after decomposing regions
    plot.plot_detection(rna_mip, spots_post_decomposition, contrast=True)

    # Clusters detection
    global spots_post_clustering
    global clusters
    spots_post_clustering, clusters = detection.detect_clusters(
        spots=spots_post_decomposition,
        voxel_size=(300, 103, 103),
        radius=450,
        nb_min_spots=2)#4

    print("detected spots after clustering")
    print("\r shape: {0}".format(spots_post_clustering.shape))
    print("\r dtype: {0}".format(spots_post_clustering.dtype), "\n")
    print("detected clusters")
    print("\r shape: {0}".format(clusters.shape))
    print("\r dtype: {0}".format(clusters.dtype))

    # Plot
    plot.plot_detection(rna_mip,
                        spots=[spots_post_decomposition, clusters[:, :3]],
                        shape=["circle", "polygon"],
                        radius=[3, 6],
                        color=["red", "blue"],
                        linewidth=[1, 2],
                        fill=[False, True],
                        contrast=True, path_output=os.path.join(path_input, f"output_images_{cell_name}"))


In [ ]:
def extract_number_from_filename(filename):
    pattern = re.compile(r'(\d{3})\.tif_C\d\.tif')
    match = re.search(pattern, filename)
    if match:
        return int(match.group(1))
    else:
        return float('inf')

#for dmso 1 digit

# def extract_number_from_filename(filename):
#     pattern = re.compile(r'(\d{3})\.tif_C\d\.tif')
#     match = re.search(pattern, filename)
#     if match:
#         # Convert the matched group to an integer
#         num = int(match.group(1))
#         # Format the number to have leading zeros and three digits
#         return f"{num:03d}"
#     else:
#         return float('inf')


In [ ]:
# Updated interactive function
def process_images_interactively(path_input):
    output_folder = os.path.join(path_input, 'output')  # Construct the path to the "output" subfolder

    os.chdir(output_folder)  # Change the working directory to the "output" subfolder

    # Guide user in the "output" subfolder
    print(f"Selected folder: {output_folder}")
    top_level_folder = os.path.basename(os.path.normpath(path_input))
    path_output = os.path.join(path_input, "output")

    if os.path.exists(path_output):
        subfolders = [f for f in os.listdir() if os.path.isdir(f)]
        print("Subfolders:")
        for i, subfolder in enumerate(subfolders):
            print(f"{i + 1}: {subfolder}")

        selected_subfolder_index = int(input("Enter the number of the subfolder you want to select: ")) - 1

        # Iterate through the subfolders
        if 0 <= selected_subfolder_index < len(subfolders):
            selected_subfolder = subfolders[selected_subfolder_index]
            os.chdir(selected_subfolder)  # Change the working directory to the selected subfolder

            # List available FISH channels with corresponding numbers
            fish_channels = [f for f in os.listdir() if "_C2.tif" in f]
            fish_numbers = [extract_number_from_filename(channel) for channel in fish_channels]
            for i, (channel, number) in enumerate(zip(fish_channels, fish_numbers)):
                print(f"{number} - {channel}")

            fish_choice = int(input("Enter the number of the FISH channel you want to select: "))

            if fish_choice in fish_numbers:
                selected_fish_channel = fish_channels[fish_numbers.index(fish_choice)]
                print(f"Selected FISH channel: {selected_fish_channel}")

                channel = 2
                treatment = top_level_folder
                print(f"Selected treatment: {treatment}")
                print(f"smFISH Path: {os.path.join(path_output, 'splitted images', selected_fish_channel)}")

                # Perform processing on the selected FISH image
                cell_name = f"cell_{fish_choice}"
                process_tiff_image(os.path.join(output_folder, selected_subfolder, selected_fish_channel), channel, cell_name)


                spot_csv_name = f"spots_{cell_name}.csv"
                cluster_csv_name = f"clusters_{cell_name}.csv"


                # Save the data to CSV files in the "notebook 5 output" subfolder
                output_subfolder = "notebook 5 output"
                os.makedirs(os.path.join(output_folder, output_subfolder), exist_ok=True)
                path_spots = os.path.join(output_folder, output_subfolder, spot_csv_name)
                path_clusters = os.path.join(output_folder, output_subfolder, cluster_csv_name)

                print(f"Saving CSV files for {fish_choice}")
                print(f"Path Spots: {path_spots}")
                print(f"Path Clusters: {path_clusters}")
                try:
                    if spots_post_clustering is not None:
                        stack.save_data_to_csv(spots_post_clustering, path_spots)
                        print(f"CSV file saved for Spots - {fish_choice}")
                    if clusters is not None:
                        stack.save_data_to_csv(clusters, path_clusters)
                        print(f"CSV file saved for Clusters - {fish_choice}")
                except Exception as e:
                    print(f"Error saving CSV files for {fish_choice}: {e}")




                # Reset the working directory back to the original path
                os.chdir(output_folder)  # Change back to the "output" directory
            else:
                print("Invalid FISH channel selection. Please try again.")
        else:
            print("Invalid channel index selected.")
    else:
        print("No 'output' folder found in the selected path. Please choose a different folder.")



if __name__ == "__main__":
    # Allow the user to choose a folder interactively in Google Colab
    path_input = choose_folder_colab()

    if path_input:
        # The user selected the treatment folder, so guide them in the "output" subfolder
        process_images_interactively(path_input)


In [ ]:
# Corrected image index, user selects treatment, and all spots are automatically predicted
def process_images_interactively(path_input):
    output_folder = os.path.join(path_input, 'output')  # Construct the path to the "output" subfolder
    os.chdir(output_folder)  # Change the working directory to the "output" subfolder

    # Guide user in the "output" subfolder
    print(f"Selected folder: {output_folder}")
    top_level_folder = os.path.basename(os.path.normpath(path_input))
    path_output = os.path.join(path_input, "output")
    if os.path.exists(path_output):
        subfolders = [f for f in os.listdir() if os.path.isdir(f)]
        print("Subfolders:")
        for i, subfolder in enumerate(subfolders):
            print(f"{i + 1}: {subfolder}")

        selected_subfolder_index = int(input("Enter the number of the subfolder you want to select: ")) - 1
        print("Before loop")
        # Iterate through the subfolders
        if 0 <= selected_subfolder_index < len(subfolders):
            print("Inside loop")
            selected_subfolder = subfolders[selected_subfolder_index]
            os.chdir(selected_subfolder)  # Change the working directory to the selected subfolder
            channel = 2
            # Iterate over all fish channels
            fish_channels = [f for f in os.listdir() if "_C2.tif" in f]

            for fish_channel in fish_channels:
                fish_number = extract_number_from_filename(fish_channel)

                print(f"Fish channel: {fish_channel}")

                treatment = top_level_folder
                print(f"Selected treatment: {treatment}")
                print(f"smFISH Path: {os.path.join(path_output, 'splitted images', fish_channel)}")
                cell_name = f"cell_{fish_channel}"
                # Perform processing on the selected FISH image
                process_tiff_image(os.path.join(output_folder, selected_subfolder, fish_channel), channel, cell_name)
                # Generate the output file names based on the selected image index and folder name
                cell_name = f"cell_{fish_number}"
                spot_csv_name = f"spots_{cell_name}.csv"
                cluster_csv_name = f"clusters_{cell_name}.csv"


                # Save the data to CSV files in the "notebook 5 output" subfolder
                output_subfolder = "notebook 5 output"
                os.makedirs(os.path.join(output_folder, output_subfolder), exist_ok=True)
                path_spots = os.path.join(output_folder, output_subfolder, spot_csv_name)
                path_clusters = os.path.join(output_folder, output_subfolder, cluster_csv_name)

                print(f"Saving CSV files for {fish_channel}")
                print(f"Path Spots: {path_spots}")
                print(f"Path Clusters: {path_clusters}")
                try:
                    if spots_post_clustering is not None:
                        stack.save_data_to_csv(spots_post_clustering, path_spots)
                        print(f"CSV file saved for Spots - {fish_channel}")
                    if clusters is not None:
                        stack.save_data_to_csv(clusters, path_clusters)
                        print(f"CSV file saved for Clusters - {fish_channel}")
                except Exception as e:
                    print(f"Error saving CSV files for {fish_channel}: {e}")

            # Reset the working directory back to the original path
            os.chdir(output_folder)  # Change back to the "output" directory
        else:
            print("Invalid channel index selected.")
    else:
        print("No 'output' folder found in the selected path. Please choose a different folder.")

if __name__ == "__main__":
    # Allow the user to choose a folder interactively in Google Colab
    path_input = choose_folder_colab()

    if path_input:
        # The user selected the treatment folder, so guide them in the "output" subfolder
        process_images_interactively(path_input)